# Lab 6

## Submission checklist¶
1. Edit this file, add, commit, and push the changes to GitHub. Do not add other files.

## Package installation

In [1]:
%%capture
import subprocess
import numpy as np
import math
from collections import Counter
!conda install --quiet --yes -c conda-forge pbzip2 zstd

## Overview and references

You will write a small bit of Python code to generate random binary and FASTA files. Your
random binary files will contain a specified percentage of zeroes and ones, and your random
FASTA files will contain either DNA or protein sequences. Using this code, you will generate a
set of random data which you will then compress using gzip and bzip2. You will generate a table
comparing the original file size, compressed file size, and run time for each algorithm on each
sequence.

### Information theory
https://en.wikipedia.org/wiki/Entropy_(information_theory)

### Compression algorithms
https://en.wikipedia.org/wiki/Gzip  
https://en.wikipedia.org/wiki/Bzip2  
https://en.wikipedia.org/wiki/Zstandard  

### Numpy
https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html  
https://docs.scipy.org/doc/numpy/reference/generated/numpy.packbits.html

### Single-letter amino acid code
https://en.wikipedia.org/wiki/Proteinogenic_amino_acid

## Background
You’re working at a biotech company that generates 1000 terabytes of data every day. In a
meeting, your boss mentions that it costs the company \\$50 per terabyte of hard disk space, and
so every 1% reduction in data that must be stored translates into a \\$500 savings per day. Your
team will get a bonus this year equal to the amount of savings you’re able to generate by
compressing the company’s data.

Incentivized by this bonus, your team gets to work determining which compression algorithm
will generate the most savings. The algorithm you choose must either be quick enough to
compress 1000 terabytes a day, or efficient enough that even if all the data isn’t compressed,
the savings are maximized by the data that you have time to compress.

## Part 1 - Simulating the data
Fortunately, you know that most of the data you’ll be compressing will be nucleic acid
sequences. Some of it, however, will be binary files, and some will be protein sequences. Start
by writing some code to simulate files containing random DNA, protein, and binary data.

Using `np.random.choice` , generate **100 megabytes** (8 bits/byte $\times$ 1024 bytes/kilobyte $\times$ 1024
kilobytes/megabyte $\times$ 100) of random data containing 100%, 90%, 80%, 70%, 60%, and 50%
zeros. Be sure to call `np.packbits` on your data before writing it to a file. For example:

In [2]:
import numpy as np
example = np.random.choice([0, 1], size=1024, replace=True, p=[0.5, 0.5])
#example

In [3]:
example = np.packbits(example)
#example

Then write this data to a file in your home directory, like this:

In [4]:
with open("example", "wb") as f:
    f.write(example)

You may notice that your kernel crashes if you try to generate and write too much data at once.
The DataHub containers only allocate 1GB of memory for JupyterHub. You will need to be
creative when generating and writing large amounts of data. Consider using `open(filename, “a”)` or `open(filename, “ab”)` to append to an existing file.

### *Implement the function below*

In [5]:
def write_random_binary(prob_zero, length, output_filename):
    
    remaining_length = length
    max_write = 8*1024*1024 #Don't write more than a megabyte at a time
    f = open(output_filename, "wb") #Delete the file if it exists
    f.close()
    f = open(output_filename, "ab")
    
    while remaining_length > 0:

        write_length = min(remaining_length, max_write)
        binary_array = np.random.choice([0,1], size=write_length, replace=True, p=[prob_zero, 1-prob_zero])
        binary_array = np.packbits(binary_array)
        f.write(binary_array)
        remaining_length -= write_length


In [6]:
probs = [1, .9, .8, .7, .6, .5]
file_name_suffix = "_binary_array"
length = 8 * 1024 * 1024 * 100

for prob in probs:
    output_file_name = str(prob)+file_name_suffix
    write_random_binary(prob, length, output_file_name)

Next, generate DNA and protein sequences **100 million** letters long and write those to your
directory. The probability distribution over nucleotides/amino acids should be uniform. To write
strings generated in Numpy to a file, you’ll have to use a slightly different command, like this:
```python
with open(“nt_seq.fa”, “w”) as f:
    f.write(“”.join(my_nt_seq))
```

### *Implement the function below*

In [7]:
def write_random_dna(probs, length, output_filename):
    
    remaining_length = length
    max_write = 1024*1024 
    f = open(output_filename, "w") #Delete the file if it exists
    f.close()
    f = open(output_filename, "a")
    alphabet = ["A", "T", "G", "C"]
    
    while remaining_length > 0:
            
        write_length = min(remaining_length, max_write)
        rand_array = np.random.choice(alphabet, size=write_length, replace=True, p=probs)
        f.write("".join(rand_array))
        remaining_length -= write_length

In [8]:
write_random_dna([.25 for i in range(4)], 100000000, "dna_array")

### *Implement the function below*

In [9]:
def write_random_protein(probs, length, output_filename):
    
    remaining_length = length
    max_write = 1024*1024 
    f = open(output_filename, "w") #Delete the file if it exists
    f.close()
    f = open(output_filename, "a")
    
    while remaining_length > 0:
            
        alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
        write_length = min(remaining_length, max_write)
        rand_array = np.random.choice(alphabet, size=write_length, replace=True, p=probs)
        f.write("".join(rand_array))
        remaining_length -= write_length

In [10]:
write_random_protein([.05 for i in range(20)], 100000000, "protein_array")

## Part 2 - Compressing the data

The time command on Linux will run whatever commands follow and report how long it took. Gzip and Bzip2 will silently compress your file and produce
a new file with the extension .gz or .bz2, respectively. The `-k` option tells them not to delete the
original file once the compression has completed. `-f` overwrites the output file if it already existed.

*On each of the files you generated
above*, run gzip, bzip, pbzip2, and zstd as follows:

In [11]:
print("100% Binary")
!time gzip -k -f 1_binary_array
!time bzip2 -k -f 1_binary_array
!time pbzip2 -k -f 1_binary_array
!time zstd -k -f 1_binary_array

print("90% Binary")
!time gzip -k -f 0.9_binary_array
!time bzip2 -k -f 0.9_binary_array
!time pbzip2 -k -f 0.9_binary_array
!time zstd -k -f 0.9_binary_array

print("80% Binary")
!time gzip -k -f 0.8_binary_array
!time bzip2 -k -f 0.8_binary_array
!time pbzip2 -k -f 0.8_binary_array
!time zstd -k -f 0.8_binary_array

print("70% Binary")
!time gzip -k -f 0.7_binary_array
!time bzip2 -k -f 0.7_binary_array
!time pbzip2 -k -f 0.7_binary_array
!time zstd -k -f 0.7_binary_array

print("60% Binary")
!time gzip -k -f 0.6_binary_array
!time bzip2 -k -f 0.6_binary_array
!time pbzip2 -k -f 0.6_binary_array
!time zstd -k -f 0.6_binary_array

print("50% Binary")
!time gzip -k -f 0.5_binary_array
!time bzip2 -k -f 0.5_binary_array
!time pbzip2 -k -f 0.5_binary_array
!time zstd -k -f 0.5_binary_array

print("DNA")
!time gzip -k -f dna_array
!time bzip2 -k -f dna_array
!time pbzip2 -k -f dna_array
!time zstd -k -f dna_array

print("Protein")
!time gzip -k -f protein_array
!time bzip2 -k -f protein_array
!time pbzip2 -k -f protein_array
!time zstd -k -f protein_array

100% Binary

real	0m1.021s
user	0m0.765s
sys	0m0.200s

real	0m1.591s
user	0m1.440s
sys	0m0.125s

real	0m0.520s
user	0m2.049s
sys	0m0.218s
1_binary_array       :  0.00%   (104857600 =>   3307 bytes, 1_binary_array.zst) 

real	0m0.164s
user	0m0.140s
sys	0m0.108s
90% Binary

real	0m25.973s
user	0m25.378s
sys	0m0.450s

real	0m20.118s
user	0m19.563s
sys	0m0.232s

real	0m7.398s
user	0m29.674s
sys	0m1.287s
0.9_binary_array     : 54.71%   (104857600 => 57371503 bytes, 0.9_binary_array.zst) 

real	0m4.394s
user	0m3.619s
sys	0m0.253s
80% Binary

real	0m20.598s
user	0m19.239s
sys	0m0.681s

real	0m29.030s
user	0m28.253s
sys	0m0.305s

real	0m8.710s
user	0m38.894s
sys	0m1.370s
0.8_binary_array     : 73.91%   (104857600 => 77499767 bytes, 0.8_binary_array.zst) 

real	0m2.819s
user	0m2.425s
sys	0m0.257s
70% Binary

real	0m9.322s
user	0m8.269s
sys	0m0.570s

real	0m34.863s
user	0m34.496s
sys	0m0.261s

real	0m7.676s
user	0m46.304s
sys	0m1.354s
0.7_binary_array     : 88.65%   (104857600 => 92953895 bytes,

In [12]:
!ls -sh *

100M 0.5_binary_array	   276K 1.69_entropy_dna.zst
101M 0.5_binary_array.bz2  980K 1.86_entropy_dna
101M 0.5_binary_array.gz   264K 1.86_entropy_dna.bz2
101M 0.5_binary_array.zst  284K 1.86_entropy_dna.gz
100M 0.6_binary_array	   292K 1.86_entropy_dna.zst
101M 0.6_binary_array.bz2  100M 1_binary_array
 98M 0.6_binary_array.gz   8.0K 1_binary_array.bz2
 98M 0.6_binary_array.zst  100K 1_binary_array.gz
100M 0.7_binary_array	   4.0K 1_binary_array.zst
 96M 0.7_binary_array.bz2   40K assignment.ipynb
 90M 0.7_binary_array.gz    96M dna_array
 89M 0.7_binary_array.zst   27M dna_array.bz2
100M 0.8_binary_array	    28M dna_array.gz
 83M 0.8_binary_array.bz2   30M dna_array.zst
 78M 0.8_binary_array.gz   4.0K example
 74M 0.8_binary_array.zst   96M protein_array
100M 0.9_binary_array	    53M protein_array.bz2
 59M 0.9_binary_array.bz2   58M protein_array.gz
 57M 0.9_binary_array.gz    64M protein_array.zst
 55M 0.9_binary_array.zst   32K random_dna_covid_length
980K 1.36_entropy_dna	   4.0K RE

Keep track of the size of the input files, the size of the output files (tip: try `!ls -sh *`), and the time each command took to run in a table such as the one below:

*Note: Numbers in table are from a different run that the one displayed in the terminal output above*

|      File                 | Input File | Output File |    Time(s) |
|---------------------------|------------|-------------|------------|
|  100% zeros, gzip          |   100MiB    |    100KiB     |    0.755   |
|  100% zeros, bzip2          |   100MiB    |    8.0KiB     |    1.036   |
|  100% zeros, pbzip2          |   100MiB    |   3.2KiB     |    0.389   |
|  100% zeros, zstd          |   100MiB    |    4.0KiB     |    0.082   |
|  90% zeros, gzip          |   100MiB    |    57MiB     |    24.294   |
|  90% zeros, bzip2          |   100MiB    |    59MiB     |    14.356   |
|  90% zeros, pbzip2          |   100MiB    |    55MiB     |    3.535   |
|  90% zeros, zstd          |   100MiB    |    55MiB     |    2.013   |
|  80% zeros, gzip          |   100MiB    |    78MiB     |    17.728   |
|  80% zeros, bzip2          |   100MiB    |    83KiB     |    16.907   |
|  80% zeros, pbzip2          |   100MiB    |    74MiB     |    4.376   |
|  80% zeros, zstd          |   100MiB    |    74KiB     |    1.886   |
|  70% zeros, gzip          |   100MiB    |    90MiB     |    7.718   |
|  70% zeros, bzip2          |   100MiB    |    96MiB     |    18.733   |
|  70% zeros, pbzip2          |   100MiB    |    89MiB     |    4.524   |
|  70% zeros, zstd          |   100MiB    |    89MiB     |    0.739   |
|  60% zeros, gzip          |   100MiB    |    98MiB     |    5.640   |
|  60% zeros, bzip2          |   100MiB    |    101MiB     |    20.900   |
|  60% zeros, pbzip2          |   100MiB    |    97MiB     |    4.857   |
|  60% zeros, zstd          |   100MiB    |    98MiB     |    0.823  |
|  50% zeros, gzip          |   100MiB    |    101MiB     |    4.770   |
|  50% zeros, bzip2          |   100MiB    |    101MiB     |    22.284   |
|  50% zeros, pbzip2          |   100MiB    |    100MiB     |    6.060   |
|  50% zeros, zstd          |   100MiB    |    101MiB     |    0.696  |
|  DNA, gzip          |   96MiB    |    28MiB     |    14.234  |
|  DNA, bzip2          |   96MiB    |    27MiB     |    11.686   |
|  DNA, pbzip2          |   96MiB    |    30MiB     |    2.785   |
|  DNA, zstd          |   96MiB    |    30MiB     |    1.174   |
|  Protein, gzip          |   96MiB    |    58MiB     |    5.411   |
|  Protein, bzip2          |   96MiB    |    53MiB     |    13.366   |
|  Protein, pbzip2          |   96MiB    |    53MiB     |    2.876   |
|  Protein, zstd          |   96MiB    |    54KiB     |    2.024  |

## Part 3 - Entropy Coding

Use the function created earlier to generate 1 million nt DNA sequence with the following
nucleotide distributions (A, T, G, C respectively): `[0.1, 0.25, 0.25, 0.4]`, `[0.1, 0.1, 0.1, 0.7]`, & `[0.5, 0.3, 0.1, 0.1]`. Then, compute the entropy of the generated sequence and compare that to the
entropy of the true distribution. Run the compression algorithms from the previous section on
each and see which tool gets closest to the ideal code in terms of bits/nucleotide.

Record the true distribution, generated distribution, Shannon entropy, and bits/nucleotide for
each compression tool in a table such as the one below:

## gzip
 |     Sequence        |    true distribution      | actual distribution      | Shannon entropy | bits/nucleotide |
 |---------------------|---------------------------|--------------------------|-----------------|-----------------|
 |        ATGC         |    0.1,0.25,0.25,0.4      |   0.1001,0.2494,0.2504,0.4002      |      1.86       |       2.27      |
 |        ATGC         |    0.1,0.1,0.1,0.7      |   0.0997,0.0999,0.0996,0.7009      |      1.36       |       1.82      |
 |        ATGC         |    0.5,0.3,0.1,0.1      |   0.5000,0.2996,0.1002,0.1001      |      1.69       |       2.14      |
 
 ## bzip2
 |     Sequence        |    true distribution      | actual distribution      | Shannon entropy | bits/nucleotide |
 |---------------------|---------------------------|--------------------------|-----------------|-----------------|
 |        ATGC         |    0.1,0.25,0.25,0.4      |   0.1001,0.2494,0.2504,0.4002      |      1.86       |       2.11      |
 |        ATGC         |    0.1,0.1,0.1,0.7      |   0.0997,0.0999,0.0996,0.7009      |      1.36       |       1.63      |
 |        ATGC         |    0.5,0.3,0.1,0.1      |   0.5000,0.2996,0.1002,0.1001      |      1.69       |       2.02      |
 
 ## pbzip2
 |     Sequence        |    true distribution      | actual distribution      | Shannon entropy | bits/nucleotide |
 |---------------------|---------------------------|--------------------------|-----------------|-----------------|
 |        ATGC         |    0.1,0.25,0.25,0.4      |   0.1001,0.2494,0.2504,0.4002      |      1.86       |       2.39      |
 |        ATGC         |    0.1,0.1,0.1,0.7      |   0.0997,0.0999,0.0996,0.7009      |      1.36       |       2.03      |
 |        ATGC         |    0.5,0.3,0.1,0.1      |   0.5000,0.2996,0.1002,0.1001      |      1.69       |       2.26      |
 
 ## zstd
 |     Sequence        |    true distribution      | actual distribution      | Shannon entropy | bits/nucleotide |
 |---------------------|---------------------------|--------------------------|-----------------|-----------------|
 |        ATGC         |    0.1,0.25,0.25,0.4      |   0.1001,0.2494,0.2504,0.4002      |      1.86       |       2.34      |
 |        ATGC         |    0.1,0.1,0.1,0.7      |   0.0997,0.0999,0.0996,0.7009      |      1.36       |       1.98      |
 |        ATGC         |    0.5,0.3,0.1,0.1      |   0.5000,0.2996,0.1002,0.1001      |      1.69       |       2.21      |

In [16]:
length = 1000000
prob_sets = [[0.1,0.25,0.25,0.4], [0.1,0.1,0.1,0.7], [0.5,0.3,0.1,0.1]]
predicted_entropies = []

def calc_entropy(prob_set): 
    entropy = 0
    for prob in prob_set:
        entropy -= prob*math.log(prob, 2)
    entropy = round(entropy, 2)
    return entropy

for prob_set in prob_sets:
    entropy = calc_entropy(prob_set)
    predicted_entropies.append(entropy)
    file_name = str(entropy) + "_entropy_dna"
    #write_random_dna(prob_set, length, file_name)

In [ ]:
#Get actual distributions and actual entropies
actual_entropies = []

for i in range(len(predicted_entropies)):
    predicted_entropy = predicted_entropies[i]
    prob_set = prob_sets[i]
    
    file_name = str(predicted_entropy) + "_entropy_dna"
    f = open(file_name, "r")
    seq = f.readlines()[0]
    
    counts = Counter(seq)
    
    chars = sum(counts.values())
    
    print(prob_set)
    actual_dist = {k: round(v / chars,4) for k, v in counts.items()}
    
    actual_entropy = calc_entropy(list(actual_dist.values()))
    actual_entropies.append(entropy)  
    
    print("Predicted Entropy", predicted_entropy)
    print(actual_dist)
    print("Actual Entropy", actual_entropy)
    print()
    f.close()

In [19]:
print("1.86 Entropy")
!time gzip -k -f 1.86_entropy_dna
!time bzip2 -k -f 1.86_entropy_dna
!time pbzip2 -k -f 1.86_entropy_dna
!time zstd -k -f 1.86_entropy_dna

print("1.36 Entropy")
!time gzip -k -f 1.36_entropy_dna
!time bzip2 -k -f 1.36_entropy_dna
!time pbzip2 -k -f 1.36_entropy_dna
!time zstd -k -f 1.36_entropy_dna

print("1.69 Entropy")
!time gzip -k -f 1.69_entropy_dna
!time bzip2 -k -f 1.69_entropy_dna
!time pbzip2 -k -f 1.69_entropy_dna
!time zstd -k -f 1.69_entropy_dna

print()

!ls -sh *

1.86 Entropy

real	0m0.190s
user	0m0.164s
sys	0m0.010s

real	0m0.164s
user	0m0.141s
sys	0m0.008s

real	0m0.159s
user	0m0.152s
sys	0m0.012s
1.86_entropy_dna     : 29.87%   (1000000 => 298737 bytes, 1.86_entropy_dna.zst) 

real	0m0.030s
user	0m0.014s
sys	0m0.008s
1.36 Entropy

real	0m0.141s
user	0m0.124s
sys	0m0.003s

real	0m0.142s
user	0m0.115s
sys	0m0.013s

real	0m0.132s
user	0m0.121s
sys	0m0.012s
1.36_entropy_dna     : 25.39%   (1000000 => 253856 bytes, 1.36_entropy_dna.zst) 

real	0m0.030s
user	0m0.015s
sys	0m0.006s
1.69 Entropy

real	0m0.169s
user	0m0.150s
sys	0m0.004s

real	0m0.176s
user	0m0.150s
sys	0m0.012s

real	0m0.158s
user	0m0.150s
sys	0m0.013s
1.69_entropy_dna     : 28.22%   (1000000 => 282173 bytes, 1.69_entropy_dna.zst) 

real	0m0.030s
user	0m0.012s
sys	0m0.008s

100M 0.5_binary_array	   276K 1.69_entropy_dna.zst
101M 0.5_binary_array.bz2  980K 1.86_entropy_dna
101M 0.5_binary_array.gz   264K 1.86_entropy_dna.bz2
101M 0.5_binary_array.zst  284K 1.86_entropy_dna.gz
100M 0.6

**Q1.** For each of the tools, research what compression schemes are being applied under the hood. Do they have any schemes in common?

Each of the algorithms uses Huffman encoding at some stage in the compression.

**Q2.** What does it mean for a compression scheme to be lossless? Are these tools lossless compression schemes?

Lossless compression means that we can perfectly reconstruct the original data from the compressed data. Each of the four algorithms used in this notebook are lossless.

**Q3.** Which algorithm achieves the best level of compression on each file type? Which
algorithm is the fastest?

| File Type | Best Compression | Fastest Compression |
| -------- | ---------------- | ------------------- |
| Binary | pbzip2 | zstd |
| DNA | bzip2 | zstd |
| Protein | bzip2 | zstd |

**Q4.** What is the difference between bzip2 and pbzip2? Do you expect one to be faster and why?

pbzip2 has been modified from bzip2 to run on multiple processors, so I would expect it to run faster.

**Q5.** How does the level of compression change as the percentage of zeros increases? Why
does this happen?

As the percentage of zeros increases, the level of compression increases as well. This is because with a higher proportion of zeros, we have a lower entropy, so it requires less information to encode.

**Q6.** What is the minimum number of bits required to store a single DNA base? What is the minimum number of bits required to store an amino acid letter? (think about information theory for these questions)

The number of bits to store a dna base is Log_2_(4) = 2.00 and for an amino acid is is Log_2_(20) = 4.32. Note that to use binary encoding for an amino acid, we would have to round up to 5 bits. 

**Q7.** In your tests, how many bits did gzip and bzip2 actually require to store your random
DNA and protein sequences?

*Bits per character*

| Sequence | gzip | bzip2 | ideal |
| -------- | ---- | ----- | ----- |
| Uniform Random DNA | 2.35 | 2.27 | 2.00 |
| Uniform Random Protein | 7.60 | 6.95 | 4.32 |

**Q8.** Do gzip and bzip2 perform well on DNA and proteins?

Not particularly. By simply doing a binary encoding we can guarantee 2 bits per nucleotide for a DNA and 5 bits per nucleotide for a protein. Both algorithms miss this mark for both sequence types.

**Q9.** Which algorithm achieves a compression limit closest to an ideal code over the non-
uniform nucleotide distributions?

bzip2 achieves the compression closest to the limit.

## Part 4 - Compressing real data
Now that you have a sense of how random data can be compressed, let’s have a look at some
real biological data: the sequence of the SARS-COV-2 genome (`sequence.txt` included in the repository).

**Q10.** A priori, do you expect to achieve better or worse compression here than random data? Why?

I would expect to achieve better compression. I'm expecting that SARS-COV-2 genome will have some natural repetitive patterns/motifs in it that the algorithms will be able to exploit. 

Now, compress the sequence using gzip, bzip2, and zstd.

In [20]:
!time gzip -k -f sequence.txt
!time bzip2 -k -f sequence.txt
!time zstd -k -f sequence.txt


real	0m0.021s
user	0m0.004s
sys	0m0.003s

real	0m0.020s
user	0m0.002s
sys	0m0.006s
sequence.txt         : 31.67%   ( 29903 =>   9470 bytes, sequence.txt.zst)     

real	0m0.011s
user	0m0.003s
sys	0m0.002s


In [21]:
f = open("sequence.txt", "r")
covid_length = len(f.readlines()[0])
print(covid_length)

In [22]:
!ls -sh *

100M 0.5_binary_array	   276K 1.69_entropy_dna.zst
101M 0.5_binary_array.bz2  980K 1.86_entropy_dna
101M 0.5_binary_array.gz   264K 1.86_entropy_dna.bz2
101M 0.5_binary_array.zst  284K 1.86_entropy_dna.gz
100M 0.6_binary_array	   292K 1.86_entropy_dna.zst
101M 0.6_binary_array.bz2  100M 1_binary_array
 98M 0.6_binary_array.gz   8.0K 1_binary_array.bz2
 98M 0.6_binary_array.zst  100K 1_binary_array.gz
100M 0.7_binary_array	   4.0K 1_binary_array.zst
 96M 0.7_binary_array.bz2   44K assignment.ipynb
 90M 0.7_binary_array.gz    96M dna_array
 89M 0.7_binary_array.zst   27M dna_array.bz2
100M 0.8_binary_array	    28M dna_array.gz
 83M 0.8_binary_array.bz2   30M dna_array.zst
 78M 0.8_binary_array.gz   4.0K example
 74M 0.8_binary_array.zst   96M protein_array
100M 0.9_binary_array	    53M protein_array.bz2
 59M 0.9_binary_array.bz2   58M protein_array.gz
 57M 0.9_binary_array.gz    54M protein_array.zst
 55M 0.9_binary_array.zst   32K random_dna_covid_length
980K 1.36_entropy_dna	   4.0K RE

**Q11.** How does the compression ratio of this file compare to random data?

| Sequence | gzip | bzip2 | zstd |
| -------- | ---- | ----- | ----- |
| Uniform Random DNA | 2.35 | 2.27 | 2.51 |
| Sars-COV-2 genome | 2.37 | 2.18 | 2.53 |

The compression ratio was smaller (though very close) on the random DNA for two out of the three algorithms (gzip and zstd). It is worth noting, however, that the smallest overall compression ratio came from bzip2 on the SARS-COV-2 genome. I will consider this to confirm my hypothesis. 

It is possible that the relatively small sequence size for the SARS-COV-2 genome (30K bases vs. 100M) leads to some inefficiencies in the file size due to the metadata required for decompression.

## Part 5 - Implementing Run Length Encoding
Now let’s implement one of the compression schemes we learned about in lecture: run length
encoding. Recall that run length encoding involves compressing stretches of identical characters
into just a character and a count. For example, the string "ACTTTGCC" would become
"1A1C3T1G2C". For this exercise, you must use native python (no external libraries).

Write a function that converts a string into a run length encoded string.

### *Implement the function below*

In [23]:
def run_length_encoding(seq):
    
    current_char = seq[0]
    current_count = 1
    new_str = []
    for i in range(1,len(seq)):
        new_char = seq[i]
        if new_char == current_char:
            current_count += 1
        else:
            new_str.append(str(current_count))
            new_str.append(str(current_char))
            current_count = 1
            current_char = new_char
    
    new_str.append(str(current_count))
    new_str.append(str(current_char))
    
    return "".join(new_str)

Then, generate a
random DNA sequence (from a uniform distribution), which is the same length as the SARS-
COV-2 genome.

In [24]:
random_dna_cov_length = write_random_dna([.25 for i in range(4)], covid_length, "random_dna_covid_length")

**Q12.** A priori, do you expect to achieve better or worse compression on than random data
than the virus genome? Why?

For the same reason as mentioned previously, I would expect to achieve better compression on the COVID data - I expect naturally occuring DNA to have more repetitive sections.

Run your RLE compression function on both sequences and calculate the compression ratio for
each.

In [7]:
f = open("random_dna_covid_length", "r")
random_rle = run_length_encoding(f.readlines()[0])
f.close()
print(len(random_rle))

f = open("sequence.txt", "r")
covid_rle = run_length_encoding(f.readlines()[0])
f.close()
print(len(covid_rle))

45106
43655


**Q13.** Which sequence is compressed more by a run length encoding scheme?

Both sequences end up with longer sequences with run length encoding than without, but the random DNA is slightly longer. (150% vs. 146% compression by string length). Though my hypothesis was technically correct, the margin was not as large as expected. This leads me to believe that the SARS-COV-2 DNA has very few repeats, which would be consistent with what we've learned about how efficient virus genomes are in storing information.

## Part 6 - Estimating compression of 1000 terabytes
Let’s make some assumptions about the contents of the data at your biotech company. Most of
the data, say 80%, is re-sequencing of genomes and plasmids that are very similar to each
other. Another 10% might be protein sequences, and the last 10% are binary microscope
images which we’ll assume follow the worst-case scenario of being completely random.

**Q14.** Given the benchmarking data you obtained in this lab, which algorithm do you propose to use
for each type of data? Provide an estimate for the fraction of space you can save using your
compression scheme. How much of a bonus do you anticipate receiving this year?

*For the sake of this analysis, I will assume that we have one machine to run compression on, similar to the way it was done in this workbook*

We will be able to achieve the best compression ratio on the DNA data, so we will start by focusing our computing time here. To encode ~100MB of DNA with our fastest algorithm, zstd, took 1.174 seconds. So to encode 800,000,000MB (800 TB) will take 8,000,000 * 1.174 = 939,200 seconds = 261 hours. We are overwhelmingly limited by time here, so will use this algorithm to compress as much as we can in a day. Assuming the same compression ratio that was achieved on the random data (which is likely conservative, as noted in previous sections) and 24 hours of computing time, we can compress 800TB * ((96-30)/96) * (24/261) = ~50TB. We will not use computing time on the protein data, which would not have been as efficient to compress, or the binary data, which would not have compressed at all. 

I expect my team's bonus this year to be 50TB * 50 dollars/TB/day * 365 days/year = $912,500.